In [1]:
import ray
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.side_channel.environment_parameters_channel import EnvironmentParametersChannel
import numpy as np

In [2]:
ray.init()

2021-01-18 14:39:39,761	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '166.104.169.102',
 'raylet_ip_address': '166.104.169.102',
 'redis_address': '166.104.169.102:21458',
 'object_store_address': '/tmp/ray/session_2021-01-18_14-39-39_187037_2758/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-01-18_14-39-39_187037_2758/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-01-18_14-39-39_187037_2758',
 'metrics_export_port': 61195,
 'node_id': '320808d5b574d93dc59e072d154c65bcc95aec07'}

In [3]:
nEnv = 4
nCPU = 4
id = 122
envs = []
nAgent = 64

for _ in range(nEnv):
    env = ray.remote(UnityEnvironment)
    actor = env.remote('./Env/Linux/Lotte08', worker_id=id+_)
    envs.append(actor)

for e in envs:
    e.reset.remote()

In [4]:

behaviorNames ='Robot?team=0'

In [26]:
@ray.remote
def getObs(env, n):
    env: UnityEnvironment
    done = [ False for i in range(n)]
    decisionStep, terminalStep = ray.get(env.get_steps.remote(behaviorNames))
    obs, tobs = decisionStep.obs[0], terminalStep.obs[0]
    reward, treward = decisionStep.reward, terminalStep.reward
    tAgentId = terminalStep.agent_id
    obsState = np.array(obs)
    for j, state in zip(tAgentId, tobs):
        obsState[j] = np.array(state)
        done[j] = True
        reward[j] = treward[k]
        k += 1
    return reward

In [27]:
import time

In [28]:
que = []

for i in range(50):
    obj_refs = []
    x = time.time()
    for _ in range(nEnv):
        obj_refs.append(getObs.remote(envs[_], nAgent))
    que.append(ray.get(obj_refs))
    inference_t = time.time()
    print("inference:{:.3f}".format(inference_t - x))

inference:0.018
inference:0.006
inference:0.005
inference:0.005
inference:0.003
inference:0.003
inference:0.004
inference:0.005
inference:0.003
inference:0.003
inference:0.003
inference:0.003
inference:0.003
inference:0.003
inference:0.003
inference:0.003
inference:0.004
inference:0.004
inference:0.004
inference:0.003
inference:0.004
inference:0.005
inference:0.004
inference:0.004
inference:0.003
inference:0.005
inference:0.003
inference:0.003
inference:0.003
inference:0.004
inference:0.004
inference:0.003
inference:0.003
inference:0.004
inference:0.003
inference:0.004
inference:0.004
inference:0.003
inference:0.003
inference:0.004
inference:0.003
inference:0.003
inference:0.003
inference:0.003
inference:0.004
inference:0.003
inference:0.004
inference:0.003
inference:0.004
inference:0.004


In [24]:
ray.get(obj_refs[0])

RayTaskError(TypeError): [36mray::getObs()[39m (pid=4198, ip=166.104.169.102)
  File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
  File "<ipython-input-16-2d1941055af6>", line 5, in getObs
TypeError: 'ray._raylet.ObjectRef' object is not iterable

In [25]:
envs[_]

Actor(UnityEnvironment,bd37d26201000000)